# Part 1: Scipp crash course and exploring data

## Getting help

- Scipp documentation is available at https://scipp.github.io/
- Join [#scipp](https://ess-eric.slack.com/archives/C01AAGCQEU8) in the ESS Slack workspace for updates, questions, and discussions.

In [ ]:
import scipp as sc
import numpy as np

## Using Jupyter

- Press `shift-return` to run a cell and move to next cell
- Press `alt-return` to run a cell, to keep focus on current cell
- If things go wrong, `Kernel > Restart kernel and clear all outputs` is often helpful.
- Jupyter will automatically display the last (and only the last) object typed in a cell

In [ ]:
a = 5
b = 4
a
b

## Scipp crash course

- `scipp` stores data in a **multi-dimensional array** with **labeled (named) dimensions**.
  This is best imagined as `numpy` arrays, without the need to memorize and keep track of dimension order.
- Each array is combined with a **physical unit** into a **variable**.
- Variables are enhanced by **coordinates**.
  Each coordinate is also a variable.
  A variable with associated coordinates is called **data array**.
- Multiple data arrays with aligned coordinates can be combined into a **dataset**.

Consider a 2-D numpy array:

In [ ]:
a = np.random.rand(2,4)
a

Scipp variables enrich this with labelled dimensions and units, for clarity and safety.
Variables can be created from numpy arrays using `sc.array`:

In [ ]:
var = sc.array(dims=['time','location'], values=a, unit='K')
var

Dimension labels are used for many operations, the simplest example is "slicing" (or cropping):

In [ ]:
var['location',2:4]

Data arrays are created from variables:

In [ ]:
time =     sc.array(dims=['time'], unit=sc.units.s, values=[20,30])
location = sc.array(dims=['location'], unit=sc.units.m, values=np.arange(4))
array =    sc.DataArray(data=var, coords={'time':time, 'location':location})
array

Scalar variables are variables with zero dimensions.
There are two ways two create these, using `sc.scalar`, or by multiplying an value by a scipp unit:

In [ ]:
windspeed = sc.scalar(1.2, unit='m/s') # see help(sc.scalar) for additional arguments
windspeed = 1.2 * sc.Unit('m/s')
windspeed

Data arrays also support **attributes** to store additional meta information:

In [ ]:
array.attrs['windspeed'] = windspeed
array

Scipp's units protect against invalid additions and subtractions:

In [ ]:
array += windspeed # will raise an exception

Data array coordinates protect against operations between incompatible data:

In [ ]:
array['location', 0:2] + array['location', 2:4] # will raise an exception

In [ ]:
array['location', 0:2] - sc.mean(array, 'location') # ok, mean over location drops location coord

## Exploring data

When working with a dataset, the first step is usually to understand what data and metadata it contains.
In this chapter we explore how scipp supports this.

We  start by loading some data, in this case measured with a prototype of the LoKI detectors at the LARMOR beamline:

In [ ]:
data = sc.io.open_hdf5(filename='loki-at-larmor.hdf5')

Note that the exercises in the following are fictional and do not represent the actual data reduction workflow.

### Step 1: Use the HTML representation to see what the loaded data contains

The HTML representation is what Jupyter displays for a scipp object.
- Take some time to explore this view and try to understand all the information (dimensions, dtypes, units, ...).
- Note that sections can be expanded, and values can shown by clicking the &#9921; icons to the right.

In [ ]:
data

### Step 2: Plot the data

Scipp objects can by created using the `plot()` method.
Alternatively `sc.plot.plot(obj)` can be used.
Since this is neutron-scattering data, we can also use the "instrument view", provided by `sc.neutron.instrument_view(obj)`.

- Plot the loaded data and familiarize yourself with the controls.
- Create the instrument view and familiarize yourself with the controls.

In [ ]:
data.plot()

In [ ]:
sc.neutron.instrument_view(data)

### Step 3: Exporing meta data

Above we saw that many attributes are scalar variables with `dtype=DataArray`.
The single value in a scalar variable is accessed using the `value` property.
Compare:

In [ ]:
data.attrs['proton_charge_by_period']

In [ ]:
data.attrs['proton_charge_by_period'].value

Exercises:
1. Find some attributes of `data` with `dtype=DataArray` and plot their `value`.
   Also try `sc.table(attr.value)` to show a table representation.
2. Find and plot a monitor.
3. Try to normalize `data` to monitor 1.
   Why does this fail?
4. Plot all the monitors on the same plot.
   Note that `sc.plot.plot()` can be used with a Python `dict` for this purpose: `sc.plot.plot({'a':something, 'b':else})`.
5. Convert all the monitors from `'tof'` to `'wavelength'` using, e.g., `mon1_wav = sc.neutron.convert(mon1, 'tof', 'wavelength')`.
6. Inspect the HTML view and note have the "unit conversion" changed the dimensions and units.
7. Re-plot all the monitors on the same plot, now in `'wavelength'`.

In [ ]:
# Data and monitor are in unit TOF, but pixels and monitors are at different position, so data is not comparable
data / data.attrs['monitor1'].value

In [ ]:
sc.plot.plot({f'monitor{i}':data.attrs[f'monitor{i}'].value for i in [1,2,3,4,5]})

In [ ]:
sc.plot.plot({f'monitor{i}':sc.neutron.convert(data.attrs[f'monitor{i}'].value,'tof','wavelength') for i in [1,2,3,4,5]})

### Step 4: Fixing metadata

Exercises:
1. The sample-position is wrong, shift the sample by `delta = sc.scalar(value=np.array([0.01,0.01,0.04]), unit=sc.units.m)`.
2. Because of an glitch in the timing system the time-of-flight has an offset of $2.3~\mu s$.
   Fix the corresponding coordinate.
3. Use the HTML view of `data` to verify that you applied the corrections/calibrations there, rather than in a copy.

### Step 5: A closer look at the data

The 2-D plot we obtain above by default is often not very enlightening.
Define:

In [ ]:
counts = sc.sum(data, 'tof')

Exercises:
1. Create a plot of `counts` and also try the instrument view.
2. How many counts are there in total, in all spectra combined?
3. Plot a single spectrum of `data` as a 1-D plot using the slicing syntax to access the spectrum.

As seen in the instrument view the detectors consist of 4 layers of tubes, each containing 7 straws.
Let us try to split up our data, so we can compare layers.
There are other (and probably better) ways to do this, but here we try to define an integer variable containing a layer index:

In [ ]:
z = sc.geometry.z(data.coords['position'])
near = sc.min(z)
far = sc.max(z)
layer = ((z-near)*400).astype(sc.dtype.int32)
layer.unit = ''
layer.plot()

Exercises:
- Change the magic parameter `400` until pixels fall cleanly into layers, either 4 layers of tubes or 12 layers of straws.
- Store `layer` as a new coord in `data`.
- Use `sc.groupby(data, group='layer').sum('spectrum')` to group spectra into layers.
- Inspect and understand the HTML view of the result.
- Plot the result.
  There are two options:
  - Use `plot` with `projection='1d'`
  - Use `sc.plot.plot` after collapsing dimensions, `sc.collapse(grouped, keep='tof')`
- Bonus: When grouping by straw layers, there is a different number of straws in the center layer of each tube (3 instead of 2) due to the flower-pattern arrangement of straws.
  Define a helper data array with data set to one for each spectrum, group by layers and sum over spectrum as above, and use this result to normalize the layer-grouped data from above to spectrum count.

In [ ]:
data.coords['layer'] = layer
grouped = sc.groupby(data, group='layer').sum('spectrum')
grouped.plot(projection='1d')
sc.plot.plot(sc.collapse(grouped, keep='tof'))

In [ ]:
norm = sc.DataArray(data=layer*0+1, coords={'layer':layer})
norm = sc.groupby(norm, group='layer').sum('spectrum')
sc.plot.plot(sc.collapse(grouped/norm, keep='tof'))

In [ ]:
#from mantid.kernel import config
#folder = '/folder/with/downloaded/files'
#config.appendDataSearchDir(folder)
#config.saveConfig(config.getUserFilename())

In [ ]:
#run_number = 49338
#data = sc.neutron.load(filename=f'LARMOR000{run_number}')
#edges = sc.array(dims=['tof'], unit='us', values=np.linspace(5.0, 100000.0, num=201))
#data = sc.rebin(data, 'tof', edges)
#data.attrs['monitor1'].value = sc.rebin(data.attrs['monitor1'].value, 'tof', edges)
#data.attrs['monitor2'].value = sc.rebin(data.attrs['monitor2'].value, 'tof', edges)
#data.attrs['monitor3'].value = sc.rebin(data.attrs['monitor3'].value, 'tof', edges)
#data.attrs['monitor4'].value = sc.rebin(data.attrs['monitor4'].value, 'tof', edges)
#data.attrs['monitor5'].value = sc.rebin(data.attrs['monitor5'].value, 'tof', edges)
#data.to_hdf5(filename='loki-at-larmor.hdf5')